<a href="https://colab.research.google.com/github/Jimmy586/Algorithm-Optimization-Technics/blob/main/MarkovChainNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly

In [1]:
!pip install opendatasets

In [27]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import opendatasets as od
import matplotlib.pyplot as plt
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

In [14]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jimmyclariel","key":"1552fd3e43f3a9df148e47914f456c44"}'}

In [31]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
od.download('https://www.kaggle.com/datasets/idevji1/sherlock-holmes-stories')

100%|██████████| 9.93M/9.93M [00:00<00:00, 81.6MB/s]

In [29]:
from IPython.utils import text
from numpy.core.multiarray import empty
from nltk.parse.chart import EmptyPredictRule
story_path = "./sherlock-holmes-stories/sherlock/sherlock/"

def read_all_stories(story_path):
  txt = []
  for _,_, files in os.walk(story_path):
    for file in files:
      with open(story_path+file) as f:
        for line in f:
          line = line.strip()
          if line=='----------': break
          if line!='':txt.append(line)
  return txt

stories = read_all_stories(story_path)

print(len(stories))

215021


# **Cleaning processes**

In [33]:
def clean_txt(text):
  cleaned_txt = []
  for line in text:
    line = line.lower()
    line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", line)
    tokens = word_tokenize(line)
    words = [word for word in tokens if word.isalpha()]

    cleaned_txt += words
  return cleaned_txt


cleaned_story = clean_txt(stories)
print("number of words = ", len(cleaned_story))


number of words =  2332247


# **Creating MARKOV MODEL**

In [39]:
def make_markov_model(cleaned_stories, n_gram=2):
  markov_model = {}
  for i in range( len(cleaned_stories)-n_gram-1):
    curr_state, next_state = "",""
    for j in range(n_gram):
      curr_state += cleaned_stories[i+j] + " "
      next_state += cleaned_stories[i+j+n_gram] + " "
    curr_state = curr_state[:-1]
    next_state = next_state[:-1]
    if curr_state not in markov_model:
      markov_model[curr_state] = {}
      markov_model[curr_state][next_state] = 1
    elif next_state in markov_model[curr_state]:
      markov_model[curr_state][next_state] +=1
    else:
      markov_model[curr_state][next_state] =1

  # calculating the transition probs
  for curr_state, transition in markov_model.items():
    total =  sum(transition.values())
    for state, count in transition.items():
      markov_model[curr_state][state] = count/total

  return markov_model

markov_model = make_markov_model(cleaned_story,2)

In [40]:
print("number of states = ", len(markov_model.keys()))

number of states =  208717


In [47]:
print(markov_model['sherlock holmes'])

{'had interested': 0.0029390154298310064, 'there were': 0.005878030859662013, 'was standing': 0.0029390154298310064, 'not so': 0.0029390154298310064, 'is free': 0.0029390154298310064, 'gazing fixedly': 0.0029390154298310064, 'in his': 0.0029390154298310064, 'with peculiar': 0.0029390154298310064, 'peters as': 0.0029390154298310064, 'im going': 0.0029390154298310064, 'said peters': 0.0029390154298310064, 'was too': 0.0029390154298310064, 'i find': 0.0029390154298310064, 'standing fully': 0.0029390154298310064, 'this is': 0.005878030859662013, 'ran her': 0.0029390154298310064, 'had been': 0.011021307861866276, 'leaning back': 0.0029390154298310064, 'returned from': 0.0029390154298310064, 'and i': 0.019103600293901544, 'as we': 0.011756061719324026, 'opened quite': 0.002204261572373255, 'i read': 0.002204261572373255, 'shrugged his': 0.00440852314474651, 'had listened': 0.005878030859662013, 'to undertake': 0.0029390154298310064, 'can return': 0.0029390154298310064, 'left the': 0.00293901

# **Generating Sherlock Holmes like stories**

In [61]:
def generate_story(markov_model, limit=100, start="dear sherlock"):
  n=0
  curr_state = start
  next_state = None
  story = curr_state + " "
  while n<limit:
    next_state =  random.choices(list(markov_model[curr_state].keys()),list(markov_model[curr_state].values()))
    curr_state = next_state[0]
    story += curr_state + " "
    n +=1

  return story

In [79]:
phrase = {"sherlock holmes","old gentleman","of detail","dear holmes","separate explanations","her voyage", "happy thought","the case"}
for word in phrase:
  print(generate_story(markov_model, start=word , limit=100))

the case i ended by doing this mark of their opponent and they hoped that time had such a day will permit me colonel i will personally conduct you no doubt travel a little forward and laid her on the clay outside and on the other side at a nine you will excuse us for feeling sore when you jump in with him and yet he shoved the large tin of cigarettes here which we had so clear if only you two days ago woodley came up to them when the church he was a man who was in the british army it did seem very loud you dont think it likely that a bright future awaits you when goods delivered pierrot then comes our expedition of by an examination of his messenger came cecil barker much excited hold it back for cargo i would have been easier but you saw me into the garden we can enter in a research into the pathology of catalepsy and finally when i saw the vague loom of two glowing eyes towards us his head had been employed between and the brother the young man but our legal work begins exactly said 